In [ ]:
#Load dataset
import pandas as pd
import numpy as np
df = pd.read_csv('merge.csv')
df.sort_values('Area Code (FAO)')
df

In [2]:
#clear rows whithout 60 years
d=(df.groupby('Area Code (FAO)').count()==60)[['Year']]
d=list(d[d['Year']==True].reset_index()['Area Code (FAO)'])
for i in range(len(df)):
    if df['Area Code (FAO)'][i] not in d:
        df.drop(i,axis=0,inplace=True)
df

,Area Code (FAO),Year,population,yield,harvested,Value
0,2,1961,8790.140,10220,2230000.0,2279000.0
1,2,1962,8969.047,9735,2341000.0,2279000.0
2,2,1963,9157.465,8317,2341000.0,1947000.0
3,2,1964,9355.514,9510,2345000.0,2230000.0
4,2,1965,9565.147,9723,2347000.0,2282000.0
...,...,...,...,...,...,...
6608,181,2016,14452.704,19013,22094.0,42008.0
6609,181,2017,14751.101,17542,22070.0,38715.0
6610,181,2018,15052.184,16206,27767.0,45000.0
6611,181,2019,15354.608,18278,43769.0,80000.0


In [3]:
(df.groupby('Area Code (FAO)').count()==60).sum()
Y=df['Value']
Y=np.array(Y)
Y=Y.reshape(83,1,60)
Y.shape

(83, 1, 60)

In [4]:
df.duplicated().sum()

0

In [5]:
arr=np.array([])

#create a 2d array for population that each rows show populations for a country
population=df.loc[:,"population"]
population=population.values.tolist()
population=np.array(population)
population=population.reshape(83,60)
print(population)


#create a 2d array for yield that each rows show yields for a country
Yield=df.loc[:,"yield"]
Yield=Yield.values.tolist()
Yield=np.array(Yield)
Yield=Yield.reshape(83,60)

#create a 2d array for harvested that each rows show harvesteds for a country
harvested=df.loc[:,"harvested"]
harvested=harvested.values.tolist()
harvested=np.array(harvested)
harvested=harvested.reshape(83,60)

#create a 2d array for production that each rows show productions for a country
#production=df.loc[:,"production"]
#production=production.values.tolist()
#production=np.array(production)
#production=production.reshape(83,60)


#create a 1d array that we will make it to a 3d array
for i in range(83):
    arr=np.append(arr,population[i])
    arr=np.append(arr,Yield[i])
    arr=np.append(arr,harvested[i])
    #arr=np.append(arr,production[i])

#reshape 1d array to a 3d array that each dimension is for a country , culomn for years , and rows for values
arr=arr.reshape(83,3,60)
arr


[[ 8790.14   8969.047  9157.465 ... 36686.784 37769.499 38972.23 ]
 [ 1769.079  1828.232  1888.302 ...  2877.013  2873.883  2866.849]
 [11598.608 11778.26  11969.451 ... 41927.007 42705.368 43451.666]
 ...
 [ 5646.668  5753.386  5860.197 ... 30790.513 31546.691 32284.046]
 [ 3219.451  3323.427  3431.381 ... 17835.893 18380.477 18927.715]
 [ 3925.952  4049.778  4177.931 ... 15052.184 15354.608 15669.666]]


array([[[8.7901400e+03, 8.9690470e+03, 9.1574650e+03, ...,
         3.6686784e+04, 3.7769499e+04, 3.8972230e+04],
        [1.0220000e+04, 9.7350000e+03, 8.3170000e+03, ...,
         2.2100000e+04, 2.0951000e+04, 1.9434000e+04],
        [2.2300000e+06, 2.3410000e+06, 2.3410000e+06, ...,
         1.6350000e+06, 2.3340000e+06, 2.6680000e+06]],

       [[1.7690790e+03, 1.8282320e+03, 1.8883020e+03, ...,
         2.8770130e+03, 2.8738830e+03, 2.8668490e+03],
        [7.7310000e+03, 1.0522000e+04, 7.1270000e+03, ...,
         3.6927000e+04, 4.0680000e+04, 4.3271000e+04],
        [1.2650000e+05, 1.3914400e+05, 8.6660000e+04, ...,
         6.5072000e+04, 5.7330000e+04, 5.3946000e+04]],

       [[1.1598608e+04, 1.1778260e+04, 1.1969451e+04, ...,
         4.1927007e+04, 4.2705368e+04, 4.3451666e+04],
        [4.0600000e+03, 8.0450000e+03, 8.3260000e+03, ...,
         2.0433000e+04, 1.9630000e+04, 1.6811000e+04],
        [1.6890640e+06, 1.8733000e+06, 1.9092030e+06, ...,
         1.9484020e+06, 1

In [6]:
#preproccessing dataset 
from sklearn.preprocessing import StandardScaler
scalers={}
for i in range(arr.shape[1]):
    scalers[i]=StandardScaler()
    arr[:,i,:] = scalers[i].fit_transform(arr[:,i,:])

In [7]:
#split dataset into train and test parts
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(arr,Y, test_size=0.2)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(66, 3, 60) (17, 3, 60) (66, 1, 60) (17, 1, 60)


In [8]:
#Reshape X_train and X_test
X_train = X_train.reshape(X_train.shape[0], 3, 60, 1)
X_test = X_test.reshape(X_test.shape[0], 3, 60, 1)

X_train.shape, X_test.shape

((66, 3, 60, 1), (17, 3, 60, 1))

In [9]:
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.6 MB/s eta 0:00:00


In [10]:
from keras.models import Sequential

from tensorflow import keras
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D,Input
from keras import layers, Model
from keras_tuner.tuners import RandomSearch

In [11]:
#create convolutional layers and fit the model 
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,5),activation='relu',input_shape=(3,60,1),padding='same'))
model.add(MaxPooling2D(padding='same'))

model.add(Conv2D(64,kernel_size=(3,5),activation='relu',padding='same'))
model.add(MaxPooling2D(padding='same'))

model.add(Flatten())
model.add(Dense(960))
model.add(Dense(60))
model.compile(loss='mean_absolute_percentage_error',metrics=['mean_absolute_percentage_error'],optimizer='adam')

model.fit(X_train,Y_train,epochs=1000,batch_size=1,validation_data=(X_test,Y_test))




Epoch 1/1000
66/66 [==============================] - 2s 11ms/step - loss: 98.6685 - mean_absolute_percentage_error: 98.6685 - val_loss: 95.9568 - val_mean_absolute_percentage_error: 95.9568
Epoch 2/1000
66/66 [==============================] - 1s 10ms/step - loss: 97.6462 - mean_absolute_percentage_error: 97.6462 - val_loss: 98.8531 - val_mean_absolute_percentage_error: 98.8531
Epoch 3/1000
66/66 [==============================] - 1s 10ms/step - loss: 97.3080 - mean_absolute_percentage_error: 97.3080 - val_loss: 97.6022 - val_mean_absolute_percentage_error: 97.6022
Epoch 4/1000
66/66 [==============================] - 1s 8ms/step - loss: 96.0737 - mean_absolute_percentage_error: 96.0737 - val_loss: 96.6537 - val_mean_absolute_percentage_error: 96.6537
Epoch 5/1000
66/66 [==============================] - 1s 9ms/step - loss: 96.8171 - mean_absolute_percentage_error: 96.8171 - val_loss: 96.7424 - val_mean_absolute_percentage_error: 96.7424
Epoch 6/1000
66/66 [===========================

# New Section

In [12]:
pip install alibi[tensorflow]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.4/472.4 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 91.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.9 MB/s eta 0:00:00


In [13]:
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(66, 3, 60, 1) (17, 3, 60, 1) (66, 1, 60) (17, 1, 60)


In [14]:
import tensorflow as tf
from alibi.explainers import IntegratedGradients
from functools import partial

target_fn=partial(np.argmax, axis=1)

ig=IntegratedGradients(model, target_fn=target_fn)
explanations=ig.explain(X_train)
predictions=model.predict(X_train)
#ig  = IntegratedGradients(model,
#                        method="gausslegendre",
#                          n_steps=20)

#predictions = model.predict(X_train)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


3/3 [==============================] - 0s 8ms/step


In [24]:
explanations.predictions

array([[1.4661005e+05, 1.5074988e+05, 1.5490923e+05, ..., 4.3458981e+05,
        3.5429728e+05, 4.3353225e+05],
       [1.5412368e+06, 2.0662274e+06, 2.2175938e+06, ..., 4.6636375e+06,
        4.7179175e+06, 5.7353950e+06],
       [5.6633275e+05, 7.3907288e+05, 8.1263112e+05, ..., 1.6145830e+06,
        1.6616092e+06, 1.9742926e+06],
       ...,
       [4.3647684e+05, 4.9383984e+05, 5.3301269e+05, ..., 1.0021241e+06,
        9.7177762e+05, 1.0958945e+06],
       [9.7191809e+02, 1.0986787e+03, 1.1741982e+03, ..., 3.1525784e+03,
        2.9749336e+03, 3.3496433e+03],
       [4.9758286e+03, 6.0645103e+03, 5.5834658e+03, ..., 5.4766787e+03,
        4.8023022e+03, 5.0941626e+03]], dtype=float32)

In [15]:
# Calculate attributions
#predictions = model(X_train).numpy()
print(predictions)

#explanation = ig.explain(X_train,
#                         baselines = None,
#                         target = None)

attributions = explanations.attributions

[[1.46610047e+05 1.50749875e+05 1.54909234e+05 ... 4.34589812e+05
  3.54297281e+05 4.33532250e+05]
 [1.54123675e+06 2.06622738e+06 2.21759375e+06 ... 4.66363750e+06
  4.71791750e+06 5.73539500e+06]
 [5.66332750e+05 7.39072875e+05 8.12631125e+05 ... 1.61458300e+06
  1.66160925e+06 1.97429262e+06]
 ...
 [4.36476844e+05 4.93839812e+05 5.33012688e+05 ... 1.00212406e+06
  9.71777750e+05 1.09589438e+06]
 [9.71918274e+02 1.09867871e+03 1.17419812e+03 ... 3.15257861e+03
  2.97493335e+03 3.34964258e+03]
 [4.97582861e+03 6.06451074e+03 5.58346533e+03 ... 5.47667920e+03
  4.80230273e+03 5.09416406e+03]]


In [17]:
for i in predictions:
  print(i)

[146610.05 150749.88 154909.23 153163.66 129094.11  96247.46 151049.92
 162611.67 151573.8  138646.23 140227.02 128730.48 139517.19 212265.36
 204967.12 250372.89 236547.53 277186.   264399.   279308.53 281223.47
 318310.06 323179.3  365812.12 369578.22 368233.   329419.66 293041.94
 275930.72 307988.44 332232.62 312515.78 311230.84 312458.5  395407.53
 413293.72 356232.53 367157.47 313396.34 342445.62 360356.6  390325.22
 412426.9  434939.34 428137.25 415463.5  406300.34 422876.62 436334.7
 452793.06 481596.62 550944.1  479316.06 507077.4  509827.53 517343.2
 437107.84 434589.8  354297.28 433532.25]
[1541236.8 2066227.4 2217593.8 2254696.  1909469.1 1712429.9 2451872.8
 2793552.5 2855744.5 2556757.  2759339.8 2539451.5 2500139.  3059031.8
 3258597.  3908173.2 3210062.8 3805718.  3651120.8 3734614.5 3815925.2
 3953897.8 4276514.5 4133092.2 4543747.5 4937459.  4690222.  4226414.
 4143328.  4811439.5 5015023.5 4636854.5 4656681.  4191438.5 4366276.5
 5225951.5 4585506.  5155331.5 4650891

In [18]:
predictions2 = model.predict(X_test)
predictions2

1/1 [==============================] - 0s 20ms/step


array([[3.9615819e+05, 6.0438300e+05, 5.9355381e+05, ..., 1.5373050e+06,
        1.4640082e+06, 1.9288414e+06],
       [6.8799344e+04, 6.7810023e+04, 7.1741352e+04, ..., 1.8203556e+05,
        1.8698109e+05, 1.8690797e+05],
       [1.7983794e+03, 1.7108405e+03, 1.9118197e+03, ..., 1.4953844e+03,
        1.2543496e+03, 1.0293135e+03],
       ...,
       [2.9601389e+03, 3.3746091e+03, 3.9243660e+03, ..., 1.0790614e+04,
        8.5025615e+03, 1.1290958e+04],
       [4.2939315e+06, 6.3279470e+06, 6.6607365e+06, ..., 1.4171518e+07,
        1.4409965e+07, 1.8125030e+07],
       [4.6349371e+02, 4.6008066e+02, 4.2826163e+02, ..., 1.6972488e+03,
        1.5505813e+03, 1.7160610e+03]], dtype=float32)

In [38]:
#Write ig result tp file
file = open('ig_train.csv','w')
for item in predictions:
	file.write(str(item)+"\n")
file.close()

file2 = open('ig_test.csv','w')
for item in predictions2:
	file2.write(str(item)+"\n")
file2.close()